In [1]:
import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from highlight_text import ax_text
from dateutil.relativedelta import relativedelta
import matplotlib as mpl
from matplotlib import image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from urllib import response
import sys
from distutils import core
import os
import matplotlib.font_manager as fm
from highlight_text import ax_text
from matplotlib.offsetbox import OffsetImage,AnnotationBbox


module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\utils")
    sys.path.append(module_path+"\\assets")

from tukan_helper_functions import get_tukan_api_request

# For creating cool charts :)
plt.style.use(module_path + '\\utils\\tukan_style.mpl')

In [2]:
path = [x for x in sys.path if "assets" in x][0] + "\\fonts\\Teko\\"

for x in os.listdir(path):
    if x.split(".")[-1] == "ttf":
        fm.fontManager.addfont(path + "/" + x)
        try:
            fm.FontProperties(weight=x.split(
                "-")[-1].split(".")[0].lower(), fname=x)
        except:
            x

path = [x for x in sys.path if "assets" in x][0] + "\\fonts\\Dosis\\"

for x in os.listdir(path):
    if x.split(".")[-1] == "ttf":
        fm.fontManager.addfont(path + "/" + x)
        try:
            fm.FontProperties(weight=x.split(
                "-")[-1].split(".")[0].lower(), fname=x)
        except:
            x

In [3]:
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "en",
    "group_by": [
        "geography"
    ],
    "categories": {
        "geography": "all"
    },
    "request": [
        {
            "table": "mex_segob_country_entries_air",
            "variables": [
                "521e1381cad0e64"
            ]
        }
    ],
    "from": "2010-01-01"
}

data = get_tukan_api_request(payload)["data"]

Success getting the data


In [4]:
payload = {
    "type": "data_table",
    "operation": "sum",
    "language": "es",
    "group_by": [
        "geography"
    ],
    "categories": {
        "geography": "all"
    },
    "request": [
        {
            "table": "mex_segob_country_entries_air",
            "variables": [
                "521e1381cad0e64"
            ]
        }
    ],
}

data_es = get_tukan_api_request(payload)["data"]
data_es = data_es[["geography", "geography__ref"]]

Success getting the data


In [5]:
data_es.rename(columns = {"geography":"geography_es"}, inplace = True)

In [6]:
data["year"] = data["date"].dt.year
data = data.groupby(["year", "geography", "geography__ref"]).sum().reset_index()

In [7]:
data = pd.merge(data, data_es, how = "left", on = "geography__ref")

In [8]:
countries = pd.read_csv(
    "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")

countries = countries[['name', 'alpha-2']]
countries.replace({"name":{"Bolivia (Plurinational State of)":"Bolivia"}}, inplace = True)

data = pd.merge(data, countries, how = "left", left_on = "geography", right_on="name")

In [9]:
total_Y = data[data['geography'] == "World"]["521e1381cad0e64"].iloc[-1] - data[data['geography'] == "Mexico"]["521e1381cad0e64"].iloc[-1]

In [10]:
total_Y_2 = data[data['geography'] == "World"]["521e1381cad0e64"].iloc[-3] - data[data['geography'] == "Mexico"]["521e1381cad0e64"].iloc[-3]

In [11]:
total_Y/total_Y_2 - 1

-0.25993163558355215

In [12]:
top_20_countries = data[data["year"] == 2021].sort_values(by = "521e1381cad0e64", ascending = False).head(60)["geography"]

In [17]:
data_summary = data[(data["geography"].isin(top_20_countries))]
data_summary = data_summary.pivot(index = ["geography", "alpha-2", "geography_es"], columns = "year", values = "521e1381cad0e64").reset_index()


In [18]:
data_summary = data_summary.sort_values(by = 2021, ascending = False)

In [19]:
# data_summary = data_summary[~data_summary["alpha-2"].isin(["MX", np.nan])]
data_summary.reset_index(drop = True, inplace = True)
data_summary.replace({"geography":{"United Kingdom of Great Britain and Northern Ireland":"United Kingdom", "Korea, Republic of": "Korea Republic", "Venezuela (Bolivarian Republic of)": "Venezuela", "Dominican Republic": "Dominican Rep."},"geography_es": {"República Dominicana": "Rep. Dominicana"}}, inplace = True)

In [16]:
def get_flag(name):
    path = "../../../assets/flags/{}.png".format(name)
    im = plt.imread(path)
    return im

def offset_image(name, ax):
    img = get_flag(name)
    im = OffsetImage(img, zoom=72./290)
    im.image.axes = ax

    ab = AnnotationBbox(im, (0, 1.15),  xybox=(5, -0.5), frameon=False,
                        xycoords='axes fraction',  boxcoords="offset points", pad=0)

    ax.add_artist(ab)

In [ ]:
    language = "es"
    index_K = 0
    K = 0
    if language == "es":
        title = f"¿Qué países están viajando a México? - ({index_K + 1})"
        source_text = "Fuente: TUKAN, SEGOB"
        subtitle = f"<Más de 14 millones de extranjeros volaron a México durante el> <2021>, cifra <25% menor> a <2019>, pero casi el doble que en <2020>.\n\nEn esta serie de gráficas exploramos cómo han cambiado las llegadas áreas a México en los últimos años para aquéllos \npaíses con mayor flujo de pasajeros hacia nuestro país.\n\n<NOTA: Valores en miles de pasajeros.>\n\n"
        alt_text_1 = "2021 vs. 2019"
        alt_text_2 = r"$\downarrow$ empresarios pesimistas"
        name_text = "geography_es"
    else:
        title = f"Which nationalities are flying into Mexico? - ({index_K + 1})"
        source_text = "Source: TUKAN, SEGOB"
        subtitle = f"<Over 14 million people flew into Mexico during> <2021>, that's <25% less> than in <2019>, but almost double than in <2020>.\n\nIn this series of charts we explore how travel to Mexico has changed over the years for those countries with most nationals flying into the \nLatin American country.\n\n<NOTE: Passenger values are presented in thousands.>"
        alt_text_1 = "2021 vs. 2019"
        alt_text_2 = r"$\downarrow$ pessimisstic executives"
        name_text = "geography"

    fig = plt.figure(figsize=(10,14), dpi = 300, facecolor="white")
    outer_gspec = mpl.gridspec.GridSpec(
        ncols = 5, nrows = 4, height_ratios = [0.025, .05, 0.015, .91], figure = fig
    )

    title_spec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 1, nrows = 1, subplot_spec=outer_gspec[0,:])
    desc_spec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 5, nrows = 1, subplot_spec=outer_gspec[1,:])
    dummy_spec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 5, nrows = 1, subplot_spec=outer_gspec[2,:])
    gspec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 3, nrows = 3, subplot_spec=outer_gspec[3,:], hspace = 0.4)

    cmap = mpl.cm.get_cmap("GnBu_r", 20)

    for i in range(2):
        ax = plt.subplot(gspec[i,:])
        Y = data_summary[range(2010,2022)].iloc[i].to_numpy()
        X = ["'" + str(x) for x in range(10,22)]
        flag = data_summary["alpha-2"].iloc[i]
        name = data_summary[name_text].iloc[i]
        
        Y_max = Y.max()
        offset_label = (Y_max/1000)/20

        ax.bar(X, (Y/1000), zorder = 3, width = 0.35, color = [cmap(i) for i in range(11,-1,-1)])
        k = 11
        for x, y in enumerate(Y):
            ax_text(
                x, (y/1000) + offset_label, s = f"{y/1000:,.0f}", ha = "center", va = "top", color = cmap(k-x), family = "Roboto", weight = "bold", size = 7
            )
            k = k - 1

        offset_image(flag, ax)
        change_2019 = Y[-1]/Y[-3] - 1
        if change_2019 < 0:
            text_aux = "-"
            color_change = "red"
        else:
            text_aux = "+"
            color_change = "green"


        ax_text(
            0.06, 1.18, s = f"<{name}> (<{text_aux}{abs(change_2019):.1%}> {alt_text_1})", ha = "left", va = "top", color = "black", family = "Roboto", weight = "normal", size = 9.5,
            highlight_textprops=[{"weight":"bold"},{"weight":"bold", "color": color_change}], annotationbbox_kw = {"xycoords":"axes fraction"}
        )

        ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) 
        ax.tick_params(axis='both', colors='gray', labelsize = 6)
        ax.spines['bottom'].set_color('gray')
        ax.grid(visible = False, axis = "x")
        ax.spines['left'].set_visible(False)

    aux_count = i + 1
    for i in range(2,3):
        Y_aux = data_summary[range(2010,2022)].iloc[2:6].to_numpy()
        Y_max = Y_aux.max()
        print(Y_max)
        offset_label = (Y_max/1000)/20
        for j in range(3):

            
            ax = plt.subplot(gspec[i,j])

            ax.set_ylim(0, Y_max/1000 + offset_label*3)

            ax.spines['left'].set_visible(False)

            ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) 

            ax.tick_params(axis='both', colors='gray', labelsize = 6)
            ax.spines['bottom'].set_color('gray')
            ax.grid(visible = False, axis = "x")

            Y = data_summary[range(2010,2022)].iloc[aux_count].to_numpy()
            X = ["'" + str(x) for x in range(10,22)]
            flag = data_summary["alpha-2"].iloc[aux_count]
            name = data_summary[name_text].iloc[aux_count]
            

            ax.bar(X, (Y/1000), zorder = 3, width = 0.35, color = [cmap(i) for i in range(11,-1,-1)])
            k = 11
            for x, y in enumerate(Y):
                ax_text(
                    x, (y/1000) + offset_label, s = f"{y/1000:,.0f}", ha = "center", va = "top", color = cmap(k-x), family = "Roboto", weight = "bold", size = 6
                )
                k = k - 1

            offset_image(flag, ax)
            change_2019 = Y[-1]/Y[-3] - 1
            if change_2019 < 0:
                text_aux = "-"
                color_change = "red"
            else:
                text_aux = "+"
                color_change = "green"


            ax_text(
                0.1, 1.18, s = f"<{name}> (<{text_aux}{abs(change_2019):.1%}> {alt_text_1})", ha = "left", va = "top", color = "black", family = "Roboto", weight = "normal", size = 8.25,
                highlight_textprops=[{"weight":"bold"},{"weight":"bold", "color":color_change}], annotationbbox_kw = {"xycoords":"axes fraction"}
            )
            ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) 

            aux_count += 1
            

    # # Ax for title
    ax_title = plt.subplot(title_spec[0])

    ax_text(x = 0.005, y = 0.85,
            s = title,
            ax = ax_title, weight = "bold", font = "Dosis", ha = "left", size = 22)

    ax_title.axis("off")

    # # Logo and source text

    fig.text(0.11, 0.08, source_text, fontsize=8)


    ax_desc = plt.subplot(desc_spec[0,:])
    ax_text(x = 0.005, y = 1.25,
        s = subtitle,
        highlight_textprops = [
        {"weight": "bold"}, {"weight": "bold"}, {"weight": "bold", "color": "red"}, {"weight": "bold"}, {"weight": "bold"}, {"weight": "bold"}],
        ax = ax_desc, font = "Roboto", ha = "left", va = "top", size = 8,  annotationbbox_kw = {"xycoords":"axes fraction"})
    ax_desc.axis("off")


    tukan_im = image.imread(module_path + "\\assets\\logo\\logo192.png")
    newax = fig.add_axes([0.8, 0.82, 0.08, 0.08], anchor="NE", zorder=1)
    newax.imshow(tukan_im)
    newax.axis("off")


    plt.savefig(
        f"who_likes_mexico_MAIN_{language}.png",
        dpi=300,
        bbox_inches="tight",
        facecolor="white",
        edgecolor="none",
        transparent=False,
    )

In [ ]:
for index_K, K in enumerate([5,14,23,32,41]):

    # Define language
    language = "en"
    if language == "es":
        title = f"¿Qué países están viajando a México? - ({index_K + 2})"
        source_text = "Fuente: TUKAN, SEGOB"
        subtitle = f"<Más de 14 millones de extranjeros volaron a México durante el> <2021>, cifra <25% menor> a <2019>, pero casi el doble que en <2020>.\n\nEn esta serie de gráficas exploramos cómo han cambiado las llegadas áreas a México en los últimos años para aquéllos \npaíses con mayor flujo de pasajeros hacia nuestro país.\n\n<NOTA: Valores en miles de pasajeros.>\n\n"
        alt_text_1 = "del total"
        alt_text_2 = r"$\downarrow$ empresarios pesimistas"
        name_text = "geography_es"
    else:
        title = f"Which nationalities are flying into Mexico? - ({index_K + 2})"
        source_text = "Source: TUKAN, SEGOB"
        subtitle = f"<Over 14 million people flew into Mexico during> <2021>, that's <25% less> than in <2019>, but almost double than in <2020>.\n\nIn this series of charts we explore how travel to Mexico has changed over the years for those countries with most nationals flying into the \nLatin American country.\n\n<NOTE: Passenger values are presented in thousands.>"
        alt_text_1 = "of total"
        alt_text_2 = r"$\downarrow$ pessimisstic executives"
        name_text = "geography"

    fig = plt.figure(figsize=(10,14), dpi = 300, facecolor="white")
    outer_gspec = mpl.gridspec.GridSpec(
        ncols = 5, nrows = 4, height_ratios = [0.025, .05, 0.015, .91], figure = fig
    )

    title_spec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 1, nrows = 1, subplot_spec=outer_gspec[0,:])
    desc_spec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 5, nrows = 1, subplot_spec=outer_gspec[1,:])
    dummy_spec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 5, nrows = 1, subplot_spec=outer_gspec[2,:])
    gspec = mpl.gridspec.GridSpecFromSubplotSpec(ncols = 3, nrows = 3, subplot_spec=outer_gspec[3,:], hspace = 0.4)

    cmap = mpl.cm.get_cmap("GnBu_r", 20)

    aux_count = K
    for i in range(3):
        Y_aux = data_summary[range(2010,2022)].iloc[K:K+9].to_numpy()
        Y_max = Y_aux.max()
        print(Y_max)
        offset_label = (Y_max/1000)/20
        for j in range(3):

            
            ax = plt.subplot(gspec[i,j])

            ax.set_ylim(0, Y_max/1000 + offset_label*3)

            ax.spines['left'].set_visible(False)

            ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) 

            ax.tick_params(axis='both', colors='gray', labelsize = 6)
            ax.spines['bottom'].set_color('gray')
            ax.grid(visible = False, axis = "x")

            Y = data_summary[range(2010,2022)].iloc[aux_count].to_numpy()
            X = ["'" + str(x) for x in range(10,22)]
            flag = data_summary["alpha-2"].iloc[aux_count]
            name = data_summary[name_text].iloc[aux_count]
            

            ax.bar(X, (Y/1000), zorder = 3, width = 0.35, color = [cmap(i) for i in range(11,-1,-1)])
            k = 11
            for x, y in enumerate(Y):
                ax_text(
                    x, (y/1000) + offset_label, s = f"{y/1000:,.0f}", ha = "center", va = "top", color = cmap(k-x), family = "Roboto", weight = "bold", size = 6
                )
                k = k - 1

            offset_image(flag, ax)
            change_2019 = Y[-1]/Y[-3] - 1
            if change_2019 < 0:
                text_aux = "-"
                color_change = "red"
            else:
                text_aux = "+"
                color_change = "green"


            ax_text(
                0.1, 1.18, s = f"<{name}> (<{text_aux}{abs(change_2019):.1%}> 2021 vs. 2019)", ha = "left", va = "top", color = "black", family = "Roboto", weight = "normal", size = 8.25,
                highlight_textprops=[{"weight":"bold"},{"weight":"bold", "color":color_change}], annotationbbox_kw = {"xycoords":"axes fraction"}
            )

            ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}")) 

            aux_count += 1
            

    # # Ax for title
    ax_title = plt.subplot(title_spec[0])

    ax_text(x = 0.005, y = 0.85,
            s = title,
            ax = ax_title, weight = "bold", font = "Dosis", ha = "left", size = 22)

    ax_title.axis("off")

    # # Logo and source text

    fig.text(0.11, 0.08, source_text, fontsize=8)


    ax_desc = plt.subplot(desc_spec[0,:])
    ax_text(x = 0.005, y = 1.25,
        s = subtitle,
        highlight_textprops = [
        {"weight": "bold"}, {"weight": "bold"}, {"weight": "bold", "color": "red"}, {"weight": "bold"}, {"weight": "bold"}, {"weight": "bold"}],
        ax = ax_desc, font = "Roboto", ha = "left", va = "top", size = 8,  annotationbbox_kw = {"xycoords":"axes fraction"})
    ax_desc.axis("off")


    tukan_im = image.imread(module_path + "\\assets\\logo\\logo192.png")
    newax = fig.add_axes([0.8, 0.82, 0.08, 0.08], anchor="NE", zorder=1)
    newax.imshow(tukan_im)
    newax.axis("off")


    plt.savefig(
        f"who_likes_mexico_{index_K}_{language}.png",
        dpi=300,
        bbox_inches="tight",
        facecolor="white",
        edgecolor="none",
        transparent=False,
    )

In [1]:
import os
vipshome = r"C:\Users\migue\Downloads\vips-dev-w64-all-8.12.2\vips-dev-8.12\bin"
os.environ['PATH'] = vipshome + ';' + os.environ['PATH']
import pyvips
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from matplotlib.offsetbox import OffsetImage,AnnotationBbox

# response = requests.get("https://hatscripts.github.io/circle-flags/flags/cn.svg")

# image = pyvips.Image.new_from_file("au.svg", dpi=300)
# image.write_to_file("x.png")
# image = plt.imread("x.png")
# image = OffsetImage(image, zoom = 

In [2]:
def svg_to_png(svg_path,png_path,dpi=72,scale=1,size=None):
    # Documentation
    # Ref: https://libvips.github.io/libvips/API/current/
    # Ref: https://libvips.github.io/pyvips/
    # Initialise
    debug=False
    import os
    os.environ['path'] += r';C:\programs\vips\vips-dev-8.11\bin'
    import pyvips
    # Get the image
    if size:
        image = pyvips.Image.new_from_file(svg_path,dpi=dpi,scale=1)
        if debug: print({field:image.get(field) for field in image.get_fields()})
        scale = size/image.get('width')
        image = image.resize(scale)
    else:
        image = pyvips.Image.new_from_file(svg_path,dpi=dpi,scale=scale)
    # Write the image
    if debug: print({field:image.get(field) for field in image.get_fields()})
    image.write_to_file(png_path)

In [3]:
path = "../../../assets/flags/"

for x in os.listdir(path):
    try:
        svg_to_png(path+x,path+x.split(".")[0]+".png", dpi = 100, scale = 0.1)
    except Exception as e:
        print(e)
        continue


unable to load from file b'../../../assets/flags/bq.svg'
  VipsForeignLoad: "../../../assets/flags/bq.svg" is not a known file format

unable to load from file b'../../../assets/flags/bv.svg'
  VipsForeignLoad: "../../../assets/flags/bv.svg" is not a known file format

unable to load from file b'../../../assets/flags/hm.svg'
  VipsForeignLoad: "../../../assets/flags/hm.svg" is not a known file format

unable to load from file b'../../../assets/flags/sh.svg'
  VipsForeignLoad: "../../../assets/flags/sh.svg" is not a known file format

unable to load from file b'../../../assets/flags/sj.svg'
  VipsForeignLoad: "../../../assets/flags/sj.svg" is not a known file format

unable to load from file b'../../../assets/flags/um.svg'
  VipsForeignLoad: "../../../assets/flags/um.svg" is not a known file format



In [35]:
path

'../../../assets/flags/'